In [2]:
!pip install pandas numpy matplotlib seaborn

!pip install networkx

!pip install nltk

!pip install wordcloud
!pip install textblob vaderSentiment
!pip install plotly pyvis bokeh

!pip install spacy transformers

  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-11.3.0-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.3 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.3 MB 8.0 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.3 MB 8.9 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.3 MB 9.1 MB/s eta 0:00:01


In [3]:
import pandas as pd
import numpy as np
import json
import re
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

True

In [7]:
!pip install chardet

In [4]:
DATASET_CHOICE = "traficogt"

if DATASET_CHOICE == "traficogt":
    filename = "traficogt.txt"
    dataset_name = "@TráficoGT"
elif DATASET_CHOICE == "tioberny":
    filename = "tioberny.txt"
    dataset_name = "@BArevalodeLeon"

In [ ]:
import chardet

tweets = []
failed_lines = 0
successful_lines = 0
with open(filename, 'rb') as file:
    raw_data = file.read()
    encoding_result = chardet.detect(raw_data)
    detected_encoding = encoding_result['encoding']

print(f"Encoding detectado: {detected_encoding}")
try:
    with open(filename, 'r', encoding=detected_encoding) as file:
        for line_num, line in enumerate(file, 1):
            line = line.strip()
            if line:
                try:
                    tweet = json.loads(line)
                    tweets.append(tweet)
                    successful_lines += 1
                except json.JSONDecodeError as e:
                    failed_lines += 1
                    if failed_lines <= 5:
                        print(f"Error en línea {line_num}: {str(e)[:100]}...")
except Exception as e:
    print(f"Error al abrir archivo: {e}")
    encodings_to_try = ['utf-8-sig', 'utf-8', 'latin-1', 'cp1252']
    
    for encoding in encodings_to_try:
        try:
            tweets = []
            with open(filename, 'r', encoding=encoding) as file:
                for line in file:
                    line = line.strip()
                    if line:
                        try:
                            tweet = json.loads(line)
                            tweets.append(tweet)
                        except json.JSONDecodeError:
                            continue
            if tweets:
                print(f"Cargado con encoding: {encoding}")
                break
        except:
            continue

print(f"Total de tweets cargados: {len(tweets)}")
print(f"Líneas procesadas exitosamente: {successful_lines}")
print(f"Líneas con errores: {failed_lines}")

if tweets:
    print("Archivo cargado correctamente")
else:
    print("No se pudieron cargar los tweets")

df_tweets = pd.DataFrame(tweets)


Encoding detectado: UTF-16
Error en línea 5605: Expecting ',' delimiter: line 1 column 4314 (char 4313)...
Total de tweets cargados: 5604
Líneas procesadas exitosamente: 5604
Líneas con errores: 1
Archivo cargado correctamente


In [9]:
def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def extract_mentions(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    return re.findall(r'@(\w+)', text.lower())

def extract_hashtags(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    return re.findall(r'#(\w+)', text.lower())

def extract_urls(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    return re.findall(r'http\S+|www\S+|https\S+', text)

In [ ]:
print("Columnas disponibles en el DataFrame:")
print(df_tweets.columns.tolist())
print(f"\nShape del DataFrame: {df_tweets.shape}")

if len(df_tweets) > 0:
    print("\nPrimeras columnas del primer tweet:")
    for col in df_tweets.columns[:10]:
        sample_value = df_tweets[col].iloc[0]
        print(f"  {col}: {type(sample_value)} - {str(sample_value)[:100]}...")
text_column = None
possible_text_columns = ['rawContent', 'text', 'full_text', 'content', 'tweet_text']

for col in possible_text_columns:
    if col in df_tweets.columns:
        text_column = col
        print(f"\nColumna de texto encontrada: '{col}'")
        break

if text_column is None:
    print("\nNo se encontró columna de texto. Columnas disponibles:")
    for col in df_tweets.columns:
        print(f"  - {col}")
    print("\nRevisar manualmente y definir text_column")


Columnas disponibles en el DataFrame:
['id', 'id_str', 'url', 'date', 'user', 'lang', 'rawContent', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'bookmarkedCount', 'conversationId', 'conversationIdStr', 'hashtags', 'cashtags', 'mentionedUsers', 'links', 'viewCount', 'retweetedTweet', 'quotedTweet', 'place', 'coordinates', 'inReplyToTweetId', 'inReplyToTweetIdStr', 'inReplyToUser', 'source', 'sourceUrl', 'sourceLabel', 'media', 'card', '_type']

Shape del DataFrame: (5604, 32)

Primeras columnas del primer tweet:
  id: <class 'numpy.int64'> - 1834236045598056867...
  id_str: <class 'str'> - 1834236045598056867...
  url: <class 'str'> - https://x.com/traficogt/status/1834236045598056867...
  date: <class 'str'> - 2024-09-12 14:22:06+00:00...
  user: <class 'dict'> - {'id': 93938886, 'id_str': '93938886', 'url': 'https://x.com/traficogt', 'username': 'traficogt', 'd...
  lang: <class 'str'> - es...
  rawContent: <class 'str'> - Es comprensible la resolución... El ruso sabe de 

In [15]:
df_tweets['text_original'] = df_tweets['rawContent']
df_tweets['text_clean'] = df_tweets['rawContent'].apply(clean_text)
df_tweets['mentions'] = df_tweets['rawContent'].apply(extract_mentions)
df_tweets['hashtags'] = df_tweets['rawContent'].apply(extract_hashtags)
df_tweets['urls'] = df_tweets['rawContent'].apply(extract_urls)
df_tweets['mention_count'] = df_tweets['mentions'].apply(len)
df_tweets['hashtag_count'] = df_tweets['hashtags'].apply(len)
df_tweets['url_count'] = df_tweets['urls'].apply(len)

print(f"Procesamiento completado:")
print(f"  - Tweets totales: {len(df_tweets)}")
print(f"  - Tweets con menciones: {(df_tweets['mention_count'] > 0).sum()}")
print(f"  - Tweets con hashtags: {(df_tweets['hashtag_count'] > 0).sum()}")
print(f"  - Tweets con URLs: {(df_tweets['url_count'] > 0).sum()}")

if len(df_tweets) > 0:
    sample_idx = 0
    print(f"\nEjemplo de procesamiento (tweet {sample_idx + 1}):")
    print(f"Original: {df_tweets['text_original'].iloc[sample_idx]}")
    print(f"Limpio: {df_tweets['text_clean'].iloc[sample_idx]}")
    print(f"Menciones: {df_tweets['mentions'].iloc[sample_idx]}")
    print(f"Hashtags: {df_tweets['hashtags'].iloc[sample_idx]}")
    print(f"URLs: {df_tweets['urls'].iloc[sample_idx]}")

Procesamiento completado:
  - Tweets totales: 5604
  - Tweets con menciones: 4502
  - Tweets con hashtags: 409
  - Tweets con URLs: 1007

Ejemplo de procesamiento (tweet 1):
Original: Es comprensible la resolución... El ruso sabe de engrasar maquinaria.
Limpio: es comprensible la resolución el ruso sabe de engrasar maquinaria
Menciones: []
Hashtags: []
URLs: []


In [16]:
def extract_user_info(tweet_data):
    user_info = {}
    if 'user' in tweet_data:
        user_data = tweet_data['user']
        user_info['user_id'] = user_data.get('id_str', user_data.get('id', 'unknown'))
        user_info['username'] = user_data.get('screen_name', 'unknown')
        user_info['display_name'] = user_data.get('name', 'unknown')
        user_info['followers_count'] = user_data.get('followers_count', 0)
        user_info['friends_count'] = user_data.get('friends_count', 0)
    else:
        user_info['user_id'] = tweet_data.get('user_id', 'unknown')
        user_info['username'] = tweet_data.get('username', 'unknown')
        user_info['display_name'] = tweet_data.get('display_name', 'unknown')
        user_info['followers_count'] = tweet_data.get('followers_count', 0)
        user_info['friends_count'] = tweet_data.get('friends_count', 0)
    return user_info

In [17]:
def extract_engagement_metrics(tweet_data):
    metrics = {}
    metrics['retweet_count'] = tweet_data.get('retweet_count', 0)
    metrics['favorite_count'] = tweet_data.get('favorite_count', 0)
    metrics['reply_count'] = tweet_data.get('reply_count', 0)
    metrics['is_retweet'] = 'retweeted_status' in tweet_data
    return metrics

In [18]:
user_info_list = [extract_user_info(tweet) for tweet in tweets]
user_df = pd.DataFrame(user_info_list)

engagement_list = [extract_engagement_metrics(tweet) for tweet in tweets]
engagement_df = pd.DataFrame(engagement_list)

df_tweets = pd.concat([df_tweets, user_df, engagement_df], axis=1)

In [19]:
original_count = len(df_tweets)
df_tweets = df_tweets[df_tweets['text_clean'].str.len() > 3]
df_tweets = df_tweets.drop_duplicates(subset=['text_clean'])
final_count = len(df_tweets)

In [20]:
df_tweets['username_normalized'] = df_tweets['username'].str.lower().str.strip()
df_tweets['mentions_normalized'] = df_tweets['mentions'].apply(
    lambda x: [mention.lower().strip() for mention in x] if x else []
)

In [21]:
stop_words_spanish = set(stopwords.words('spanish'))
stop_words_english = set(stopwords.words('english'))
stop_words = stop_words_spanish.union(stop_words_english)

def remove_stopwords(text):
    if not text:
        return ""
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(filtered_tokens)

df_tweets['text_processed'] = df_tweets['text_clean'].apply(remove_stopwords)

In [ ]:
interactions_list = []

for idx, row in df_tweets.iterrows():
    source_user = row['username_normalized']

    if row['is_retweet']:
        interactions_list.append({
            'source': source_user,
            'target': 'retweeted_user',
            'interaction_type': 'retweet',
            'tweet_id': idx
        })
    for mention in row['mentions_normalized']:
        interactions_list.append({
            'source': source_user,
            'target': mention,
            'interaction_type': 'mention',
            'tweet_id': idx
        })

interactions_df = pd.DataFrame(interactions_list)

In [26]:
!pip install scipy

  Using cached scipy-1.16.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.16.1-cp311-cp311-win_amd64.whl (38.6 MB)


In [27]:
if len(interactions_df) > 0:
    G = nx.from_pandas_edgelist(
        interactions_df, 
        source='source', 
        target='target', 
        create_using=nx.DiGraph(),
        edge_attr='interaction_type'
    )
    
    adjacency_matrix = nx.adjacency_matrix(G)
    
    nodes = list(G.nodes())
    adjacency_df = pd.DataFrame(
        adjacency_matrix.toarray(), 
        index=nodes, 
        columns=nodes
    )

In [28]:
print(f"Dataset seleccionado: {dataset_name}")
print(f"Tweets originales: {original_count}")
print(f"Tweets después de limpieza: {final_count}")
print(f"Tweets eliminados: {original_count - final_count}")
print(f"Usuarios únicos: {df_tweets['username_normalized'].nunique()}")
print(f"Total de interacciones: {len(interactions_df)}")
print(f"Nodos en la red: {G.number_of_nodes()}")
print(f"Aristas en la red: {G.number_of_edges()}")

df_tweets.head()

Dataset seleccionado: @TráficoGT
Tweets originales: 5604
Tweets después de limpieza: 5144
Tweets eliminados: 460
Usuarios únicos: 1
Total de interacciones: 9765
Nodos en la red: 1037
Aristas en la red: 1036


,id,id_str,url,date,user,lang,rawContent,replyCount,retweetCount,likeCount,...,display_name,followers_count,friends_count,retweet_count,favorite_count,reply_count,is_retweet,username_normalized,mentions_normalized,text_processed
0,1834236045598056867,1834236045598056867,https://x.com/traficogt/status/183423604559805...,2024-09-12 14:22:06+00:00,"{'id': 93938886, 'id_str': '93938886', 'url': ...",es,Es comprensible la resolución... El ruso sabe ...,0,0,1,...,unknown,0,0,0,0,0,False,unknown,[],comprensible resolución ruso sabe engrasar maq...
1,1834029142565658846,1834029142565658846,https://x.com/monymmorales/status/183402914256...,2024-09-12 00:39:56+00:00,"{'id': 976875408, 'id_str': '976875408', 'url'...",es,La corrupción de la @CC_Guatemala\nes descarad...,0,56,84,...,unknown,0,0,0,0,0,False,unknown,[cc_guatemala],corrupción descarada falsificación documentos ...
4,1833665391698092330,1833665391698092330,https://x.com/CubReserva/status/18336653916980...,2024-09-11 00:34:31+00:00,"{'id': 1155617398675988481, 'id_str': '1155617...",es,@soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...,0,0,1,...,unknown,0,0,0,0,0,False,unknown,"[soy_502, aztecanoticiagt, conapgt, denunciaem...",urgente zona deterioro tala inmoderada tráfico...
5,1833664000220893559,1833664000220893559,https://x.com/Alejandro2024gt/status/183366400...,2024-09-11 00:29:00+00:00,"{'id': 730220333576282112, 'id_str': '73022033...",es,@amilcarmontejo @PMTMuniGuate @Noti7Guatemala ...,0,0,0,...,unknown,0,0,0,0,0,False,unknown,"[amilcarmontejo, pmtmuniguate, noti7guatemala,...",avenidas calles avenida calle luz semáforo age...
6,1833611884664156200,1833611884664156200,https://x.com/JulyPonce21/status/1833611884664...,2024-09-10 21:01:54+00:00,"{'id': 3012525020, 'id_str': '3012525020', 'ur...",es,Favor compartir \nEl vive el zona 7 Bethania ...,0,0,0,...,unknown,0,0,0,0,0,False,unknown,"[traficogt, amilcarmontejo, pncdeguatemala]",favor compartir vive zona bethania
